In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, pipeline
from huggingface_hub import hf_hub_download
import pandas as pd

**Datasets:**
- https://huggingface.co/datasets/yahma/alpaca-cleaned
- https://huggingface.co/datasets/ccdv/arxiv-summarization
- https://huggingface.co/datasets/HuggingFaceH4/no_robots
- https://huggingface.co/datasets/Open-Orca/SlimOrca?row=0

**You can also look**: https://huggingface.co/datasets?task_categories=task_categories:text-generation&sort=trending 

**Codes:**
- https://colab.research.google.com/drive/1kEV6i96AQ94xTCvSd11TxkEaksTb5o3U?usp=sharing#scrollTo=Dg9DRDL0jv7e
- https://huggingface.co/Intel/neural-chat-7b-v3-1


**Models:**
- Gemma 7b
- Mistral 7b
- Llama-2 7b
- TinyLlama
- DPO - Zephyr